> 이 문서의 폰트크기는 브라우저 인쇄시 인쇄 배율 80%로 최적화 되어 있음

# MNIST 분류용 FNN의 Inference 파트 (Simoid 함수 사용)
> 1. 신경망 구현 보조 함수 정의
>
>

    - 개요 : MNIST 테스트 및 훈련 데이터의 예측 정확도 계산
    - 아키텍처 : 사전에 훈련된 가중치/편향 파일(pickle 형식)을 사용
      - 사전훈련된 가중치 업로드 세팅 : ch4_40_fnn_mnist_sig.pkl
                            (훈련시 가중치는 표준편차 0.1의 Gauss 분포)
      - 입력층(784)
      - 계층1(50)  : 은닉충, Sigmoid 함수 사용
      - 계층2(10)  : 출력층, Sigmoid 함수 사용
      - 소프트맥스
    - 동작모델 : 샘플모드(각 샘플별 역전파)와 배치모드 각각 구현

In [23]:
# multi line 출력을 위한 셀 -- 필요없으면 아래 두 라인을 주석(#) 처리해서 기능 해제
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [24]:
# 팝업 창을 띠우지 않고 주피터 노트북 안에 그림을 그리라는 명령
%matplotlib inline  

import numpy as np
from mnist.mnist import load_mnist
import pickle

# 1. 신경망 구현 보조 함수 정의

In [25]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    if x.ndim == 2:
        x = x - np.max(x, axis=1).reshape(-1, 1)
        x = np.exp(x) / np.sum(np.exp(x), axis=1).reshape(-1, 1)
    else:
        x = x - np.max(x)  # 오버플로 대책
        x = np.exp(x) / np.sum(np.exp(x))

    return x

# 2. 신경망 모델 생성 및 추론 관련 함수

### 네트워크 생성 함수
- 학습된 신경망의 가중치/편향을 신경망(net)에 업로드

In [26]:
def setup_network():
    with open("52-fnn-mnist-sig.pkl", 'rb') as f:
        net = pickle.load(f)
    return net

test_net = setup_network()
W1 = test_net['W1']; 
print(W1)

[[-0.00391574  0.00235982  0.15405002 ...  0.07643579  0.04048303
   0.17894486]
 [ 0.1048934  -0.03625136 -0.07409769 ...  0.15310726  0.07670112
   0.04780273]
 [-0.23270108 -0.02838503  0.03953473 ... -0.06615517 -0.0363144
   0.1027033 ]
 ...
 [-0.05065644  0.04485779 -0.16198439 ... -0.06044266 -0.01613528
   0.02149604]
 [-0.13772163  0.02926289  0.03767156 ... -0.15479492 -0.02446951
  -0.02706205]
 [-0.03207722 -0.03754213  0.14087992 ... -0.02328948  0.19048743
  -0.03575524]]


###  추론 함수

In [27]:
def setup_network():
    with open("52-fnn-mnist-sig.pkl", 'rb') as f:
        net = pickle.load(f)
        #W1, W2 = net['W1'], net['W2']
        #b1, b2 = net['b1'], net['b2']
        #print(W1)
    return net

def predict(net, x):
    W1, W2 = net['W1'], net['W2']
    b1, b2 = net['b1'], net['b2']

    v1 = np.dot(x, W1) + b1
    h1 = sigmoid(v1)

    #print("h1 :", h1)
    v2 = np.dot(h1, W2) + b2
    h2 = sigmoid(v2)
    #print("h2 :", h2)

    y = softmax(h2)
    #print("y :", y)

    return y

In [28]:
def predict(net, x):
    W1, W2 = net['W1'], net['W2']
    b1, b2 = net['b1'], net['b2']

    v1 = np.dot(x, W1) + b1
    h1 = sigmoid(v1)

    v2 = np.dot(h1, W2) + b2
    h2 = sigmoid(v2)

    y = softmax(h2)

    return y

# 3. 메인 로직

## 3.1 MNIST 데이터셋 로딩 및 신경망 생성

In [29]:
# MNIST 훈련 및 시험 데이터 적재
(x_train, d_train), (x_test, d_test) = load_mnist(normalize=True,
                                                  flatten=True,
                                                  one_hot_label=False)
# 네트워크 생성
FNN = setup_network()

## 3.2 단일 샘플 모드 추론

In [30]:
# ------ 훈련 데이터에 대한 정확도 계산
accuracy_counter = 0
for i in range(len(x_train)):
    y = predict(FNN, x_train[i])
    #print(y)
    category = np.argmax(y)     # 가장 큰 확률를 갖는 숫자
    if category == d_train[i]:
        accuracy_counter += 1

print(" 훈련 데이터 정확도: %s" % (float(accuracy_counter) / len(x_train)) )

# ------  테스트 데이터에 대한 정확도 계산
accuracy_counter = 0
for i in range(len(x_test)):
    y = predict(FNN, x_test[i])
    category = np.argmax(y)     # 가장 큰 확률를 갖는 숫자
    if category == d_test[i]:
        accuracy_counter += 1

print(" 테스트 데이터 정확도 : %s" % (float(accuracy_counter) / len(x_test)) )

 훈련 데이터 정확도: 0.87725
 테스트 데이터 정확도 : 0.8836


## 3.3 배치 모드 추론

In [31]:
# ------ 훈련 데이터에 대한 정확도 계산
BATCH_SIZE = 100 # 배치 크기
accuracy_counter = 0

for i in range(0, len(x_train), BATCH_SIZE):
    x_batch = x_train[i : i + BATCH_SIZE]
    y_batch = predict(FNN, x_batch)
    category = np.argmax(y_batch, axis=1)
    accuracy_counter += np.sum(category == d_train[i : i+BATCH_SIZE])

print(" 훈련데이터 정확도 : %s" % (float(accuracy_counter) / len(x_train)) )

# ------  테스트 데이터에 대한 정확도 계산
BATCH_SIZE = 100 # 배치 크기
accuracy_counter = 0

for i in range(0, len(x_test), BATCH_SIZE):
    x_batch = x_test[i:i+BATCH_SIZE]
    y_batch = predict(FNN, x_batch)
    category = np.argmax(y_batch, axis=1)
    accuracy_counter += np.sum(category == d_test[i:i+BATCH_SIZE])

print(" 시험 데이터 정확도 : %s" % (float(accuracy_counter) / len(x_test)))

 훈련데이터 정확도 : 0.87725
 시험 데이터 정확도 : 0.8836
